In [1]:
#### Using Finesse, Calculate the contrast defect and the required ring heater power neccesary to correct
#### for the discrepancy between the radii of curvature.

In [2]:
import pykat
from pykat.ifo import aligo
import pykat.ifo.aligo.plot
import matplotlib.pyplot as plt
import numpy as np

#### My functions
import trace_mode

                                              ..-
    PyKat 1.1.244         _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,  
   `^-....____:   +.      www.gwoptics.org/pykat



In [3]:
katfile = "aLIGO_IFO_AWC_tuning_DCoff_maxtem2.kat"

In [4]:
def calc_CD(overlap):
    CD = (1.0 - np.real(overlap) )/ 2.0
    return CD

In [5]:
class ifo_cavs(object):
    """ an IFO object that calculates the q for the various cavities in at three points
    ITMX, ITMY, and the Beamsplitter HR surface
    
    This will allow us to calculate the beam overlaps for the various modes.
    """
    def _init_(self, kat):
        None
        
    def calc_qs(self,kat):
        #### Turn off all the cav commands except XARM to extract the eigenmode at the BS
        [q_XARM_BS_x,q_XARM_BS_y] = trace_mode.from_cav_to_BS(kat,'cavXARM')

        #### Turn off all the cav commands except YARM to extract the eigenmode at the BS
        [q_YARM_BS_x,q_YARM_BS_y] = trace_mode.from_cav_to_BS(kat,'cavYARM')

        #### Turn off all cav commands except XARM and extrat mode at HR surface of ITMX
        [q_XARM_ITM_x,q_XARM_ITM_y]  = trace_mode.from_cav_to_node(kat,'cavXARM','nITMX2')

        #### Turn off all cav commands except YARM and extrat mode at HR surface of ITMY
        [q_YARM_ITM_x, q_YARM_ITM_y]  = trace_mode.from_cav_to_node(kat,'cavYARM','nITMY2')

        #### Turn off all cav commands except IMC and extrat mode at HR surface of ITMX
        [q_IMC_ITMX_x, q_IMC_ITMX_y]  = trace_mode.from_cav_to_node(kat,'cavIMC','nITMX2')

        #### Turn off all cav commands except IMC and extrat mode at HR surface of ITMY
        [q_IMC_ITMY_x, q_IMC_ITMY_y]  = trace_mode.from_cav_to_node(kat,'cavIMC','nITMY2')

         #### Turn off all cav commands except IMC and extrat mode at HR surface of ITMX
        [q_IMC_BS_x, q_IMC_BS_y]  = trace_mode.from_cav_to_node(kat,'cavIMC','nITMX2')

        #### Turn off all cav commands except PRX and extrat mode at HR surface of ITMX
        [q_PRX_ITMX_x,q_PRX_ITMX_y]  = trace_mode.from_cav_to_node(kat,'cavPRX','nITMX2')

        #### Turn off all cav commands except PRX and extrat mode at HR surface of ITMY
        [q_PRX_ITMY_x, q_PRX_ITMY_y]  = trace_mode.from_cav_to_node(kat,'cavPRX','nITMY2')
        
        #### Turn off all cav commands except PRX and extrat mode at HR surface of ITMX
        [q_PRY_ITMX_x,q_PRY_ITMX_y]  = trace_mode.from_cav_to_node(kat,'cavPRY','nITMX2')

        #### Turn off all cav commands except PRX and extrat mode at HR surface of ITMY
        [q_PRY_ITMY_x, q_PRY_ITMY_y]  = trace_mode.from_cav_to_node(kat,'cavPRY','nITMY2')
        
        q_dict = {"q_XARM_BS_x":q_XARM_BS_x,
                  "q_XARM_BS_y":q_XARM_BS_y,
                  "q_YARM_BS_x":q_YARM_BS_x,
                  "q_YARM_BS_y":q_YARM_BS_y,

                  "q_XARM_ITM_x":q_XARM_ITM_x,
                  "q_XARM_ITM_y":q_XARM_ITM_y,
                  "q_YARM_ITM_x":q_YARM_ITM_x,
                  "q_YARM_ITM_y":q_YARM_ITM_y,
                  
                  "q_IMC_ITMX_x":q_IMC_ITMX_x,
                  "q_IMC_ITMX_y":q_IMC_ITMX_y,
                  "q_IMC_ITMY_x":q_IMC_ITMY_x,
                  "q_IMC_ITMY_y":q_IMC_ITMY_y,
                
                  "q_PRX_ITMX_x":q_PRX_ITMX_x,
                  "q_PRX_ITMX_y":q_PRX_ITMX_y,
                  "q_PRX_ITMY_x":q_PRX_ITMY_x,
                  "q_PRX_ITMY_y":q_PRX_ITMY_y,
                
                  "q_PRY_ITMX_x":q_PRY_ITMX_x,
                  "q_PRY_ITMX_y":q_PRY_ITMX_y,
                  "q_PRY_ITMY_x":q_PRY_ITMY_x,
                  "q_PRY_ITMY_y":q_PRY_ITMY_y,
                
                  "q_IMC_BS_x":q_IMC_BS_x,
                  "q_IMC_BS_y":q_IMC_BS_y,
                 }
        
        return q_dict

In [6]:
#### Get the amplitude overlap between two gaussian beams
def amp_ovl(q1,q2):
    a = 2 * 1j * np.sqrt(np.imag(q1) * np.imag(q2)) / (q1-np.conjugate(q2))
    return a

In [7]:
### Calculate the required thermal power to change curvature
def calc_P(calib, R_new, R_old):
    
    P = 1/calib * ( 1/(R_new) - (1/R_old))
    
    return P

In [8]:
basekat = pykat.finesse.kat() 
basekat.load(katfile)
basekat.parse("noxaxis")
basekat.parse("maxtem "+str(2))
basekat.parse("""
ad SB9Mhz_plus 0 0 9.099471M nITMX2
ad SB9Mhz_minus 0 0 -9.099471M nITMX2

ad SB45Mhz_plus 0 0 45.497355M nITMX2
ad SB45Mhz_minus 0 0 -45.497355M nITMX2

bp ITMX_q_x x q nITMX2
bp ITMX_q_y y q nITMX2

bp ITMY_q_x x q nITMY2
bp ITMY_q_y y q nITMY2

pd REFL_port nPRBS

pd AS_port nSRBS

yaxis re:im
""")
kat=basekat.deepcopy() #copy kat code
kat.verbose = False
out = kat.run()

In [9]:
#### Set up the before parameters
kat.ITMXHR.Rcx.value = -1939.9
kat.ITMXHR.Rcy.value = -1939.9

kat.ITMYHR.Rcx.value = -1939.2
kat.ITMYHR.Rcy.value = -1939.2

kat.ETMXHR.Rcx.value = 2241.54 
kat.ETMXHR.Rcy.value = 2241.54 

kat.ETMYHR.Rcx.value = 2238.9
kat.ETMYHR.Rcy.value = 2238.9

Before = ifo_cavs().calc_qs(kat)

In [385]:
#### Set up the after parameters
kat1=basekat.deepcopy()
kat1.verbose=False

kat1.ITMXHR.Rcx.value = -1940.3
kat1.ITMXHR.Rcy.value = -1940.3

kat1.ITMYHR.Rcx.value = -1939.2
kat1.ITMYHR.Rcy.value = -1939.2

kat1.ETMXHR.Rcx.value = 2244.2 
kat1.ETMXHR.Rcy.value = 2244.2 

kat1.ETMYHR.Rcx.value = 2246.9
kat1.ETMYHR.Rcy.value = 2246.9

After = ifo_cavs().calc_qs(kat1)

In [386]:
before_overlap_ITM = amp_ovl(Before['q_XARM_ITM_x'],Before['q_YARM_ITM_x'])
before_overlap_BS = amp_ovl(Before['q_XARM_BS_x'],Before['q_YARM_BS_x'])

after_overlap_ITM = amp_ovl(After['q_XARM_ITM_x'],After['q_YARM_ITM_x'])
after_overlap_BS = amp_ovl(After['q_XARM_BS_x'],After['q_YARM_BS_x'])

In [387]:
### Contrast defect at the ITMs with resonating arms
CD_old_ITM =calc_CD(before_overlap_ITM)
CD_new_ITM =calc_CD(after_overlap_ITM)

print(CD_old_ITM, CD_new_ITM)


RH_calib = .79e-6 #diopters per watt T1400685
P_RH_Arms_resonant = calc_P(RH_calib, -1940,-1939.2)
print(P_RH_Arms_resonant)

5.934299324494852e-06 3.865230617516868e-06
0.2691773810030645


In [388]:
### Contrast defect at the BS (includes Schnupp Asymmetry and Thermal lens) with resonating arms
CD_old_BS = calc_CD(before_overlap_BS)
CD_new_BS = calc_CD(after_overlap_BS)
print(CD_old_BS, CD_new_BS)

5.683696534086735e-06 4.5933131715414355e-06


In [389]:
### Propogate only the IMC Mode to the ITMs 
IMC_overlap_ITM = pykat.BeamParam.overlap(After['q_IMC_ITMX_x'],After['q_IMC_ITMY_y'])
IMC_CD = calc_CD(IMC_overlap_ITM)

#Modal CD for single bounce mode
print(IMC_CD*1e6)

R_IMC_ITMX = 1/np.real(1/After['q_IMC_ITMX_x'])
R_IMC_ITMY = 1/np.real(1/After['q_IMC_ITMY_x'])
P_IMC_mode = calc_P(RH_calib,R_IMC_ITMY,R_IMC_ITMX)
print(R_IMC_ITMX,R_IMC_ITMY,P_IMC_mode)

160.91119351957505
-1921.5105534290692 -1922.0134399380925 0.17236286304909376


In [390]:
### Propogate only the PRX to the ITMX, and PRY to ITMY, this should estimate the overlap of the DRMI setup
PRC_overlap_ITM = pykat.BeamParam.overlap(After['q_PRX_ITMX_x'],After['q_PRY_ITMY_y'])

### Modal CD for DRMI
DRMI_CD = calc_CD(PRC_overlap_ITM)
print(DRMI_CD*1e6)

186.45815364726204


In [434]:
#### Set up the after co2 parameters
kat2=basekat.deepcopy()
kat2.verbose=False

kat2.ITMXHR.Rcx.value = -1940.3
kat2.ITMXHR.Rcy.value = -1940.3

kat2.ITMYHR.Rcx.value = -1940.3
kat2.ITMYHR.Rcy.value = -1940.3

#kat2.ITMYHR.Rcx.value = -1939.2
#kat2.ITMYHR.Rcy.value = -1939.2

kat2.ETMXHR.Rcx.value = 2244.2 
kat2.ETMXHR.Rcy.value = 2244.2 

kat2.ETMYHR.Rcx.value = 2246.9
kat2.ETMYHR.Rcy.value = 2246.9

kat2.ITMXTL.f = 43300.0 # Guess

After_co2 = ifo_cavs().calc_qs(kat2)

In [424]:
### Propogate only the IMC Mode to the ITMs 
IMC_overlap_ITM = pykat.BeamParam.overlap(After_co2['q_IMC_ITMX_x'],After_co2['q_IMC_ITMY_y'])
IMC_CD = calc_CD(IMC_overlap_ITM)

#Modal CD for single bounce mode
print(IMC_CD*1e6)

R_IMC_ITMX = 1/np.real(1/After_co2['q_IMC_ITMX_x'])
R_IMC_ITMY = 1/np.real(1/After_co2['q_IMC_ITMY_x'])
P_IMC_mode = calc_P(RH_calib,R_IMC_ITMY,R_IMC_ITMX)
print(R_IMC_ITMX,R_IMC_ITMY,P_IMC_mode)

56.12644799263266
-1944.9033632799972 -1921.5279719847917 -7.9174815490305255


In [438]:
### Propogate only the PRX to the ITMX, and PRY to ITMY, this should estimate the overlap of the DRMI setup
PRC_overlap_ITM_co2 = pykat.BeamParam.overlap(After_co2['q_PRX_ITMX_x'],After_co2['q_PRY_ITMY_y'])

### Modal CD for DRMI
DRMI_CD_co2 = calc_CD(PRC_overlap_ITM_co2)
print("Contrast Defect (PPM) is " + str(DRMI_CD_co2*1e6))

co2_calib = 62.3e-6 #diopters per watt

P_DRMI = calc_P(co2_calib, kat2.ITMXTL.f.value,kat1.ITMXTL.f.value)
print("Required Co2 power "+ str(P_DRMI))

Contrast Defect (PPM) is 0.9284982430135535
Required Co2 power -0.09455568257893743
